In [1]:
import os
import sys
dir_parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(dir_parent) # add parent directory to path
import json
import winsound
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
from tqdm import tqdm
from src.db import DataBase
from src.utils_beeps import beeps
from src.utils_date import add_days
from src.utils_stocks import get_ls_sym
from src.utils_stocks import get_df_prices
from src.utils_stocks import suppress_stdout
dir_db = os.path.join(dir_parent, 'data', 'db')
dir_db_demo = os.path.join(dir_parent, 'data', 'demo',)

ls_init_str = [
    #prices_m
    '''CREATE TABLE IF NOT EXISTS prices_m(
        sym TEXT
        ,datetime TEXT
        ,open REAL
        ,high REAL
        ,low REAL
        ,adj_close REAL
        ,volume INTEGER
        ,is_reg_hours INTEGER)''',
    
    '''CREATE INDEX IF NOT EXISTS index_prices_m_all
        ON prices_m(sym, date(datetime), is_reg_hours)''',

    '''CREATE INDEX IF NOT EXISTS index_prices_m_date
        ON prices_m(date(datetime))''',

    #prices_d
    '''CREATE TABLE IF NOT EXISTS prices_d(
        sym TEXT
        ,date TEXT
        ,open REAL
        ,high REAL
        ,low REAL
        ,adj_close REAL
        ,volume INTEGER)''',
   
    '''CREATE INDEX IF NOT EXISTS index_prices_d_date
        ON prices_d(sym, date(date))''',
    
    #stocks
    '''CREATE TABLE IF NOT EXISTS stocks(
        sym TEXT
        ,long_name TEXT
        ,sec TEXT
        ,ind TEXT
        ,quote_type TEXT
        ,fund_family TEXT
        ,summary TEXT
        ,timestamp TEXT)''',

    '''CREATE INDEX IF NOT EXISTS index_stocks
        ON stocks(sym, quote_type)''',
    
    #stocks_error
    '''CREATE TABLE IF NOT EXISTS stocks_error(
        sym TEXT)''',
    
    #proba
    '''CREATE TABLE IF NOT EXISTS proba(
        sym TEXT
        ,datetime TEXT
        ,my_index INTEGER
        ,proba REAL
        ,datetime_update TEXT)''',
]

db = DataBase(ls_init_str, dir_db)
db_demo = DataBase(ls_init_str, dir_db_demo)

# Get Misc Dataframes

In [5]:
q = '''
select
    sym
    ,count(*)
from
    prices_m
where
    date(datetime) = '2021-02-18'
group by
    sym
order by
    count(*) desc
limit 3
'''
df = pd.read_sql(q, db.conn)
df

,sym,count(*)
0,TSLA,960
1,TLRY,960
2,SNDL,960


# View Tables, Indexes

In [6]:
q = '''
SELECT *
  FROM sqlite_master
'''
pd.read_sql(q, db.conn)

,type,name,tbl_name,rootpage,sql
0,table,prices_m,prices_m,2,"CREATE TABLE ""prices_m""(\n sym TEXT\n ..."
1,table,prices_d,prices_d,397751,"CREATE TABLE ""prices_d""(\n sym TEXT\n ..."
2,table,stocks,stocks,1489407,CREATE TABLE stocks(\n sym TEXT\n ...
3,table,stocks_error,stocks_error,1489571,CREATE TABLE stocks_error(\n sym TEXT)
4,index,index_stocks,stocks,1489230,CREATE INDEX index_stocks\n ON stocks(s...
5,index,index_prices_d_date,prices_d,397754,CREATE INDEX index_prices_d_date\n ON p...
6,index,index_prices_m_all,prices_m,257721,CREATE INDEX index_prices_m_all\n ON pr...
7,index,index_prices_m_date,prices_m,1186013,CREATE INDEX index_prices_m_date\n ON p...
8,table,proba,proba,2229931,CREATE TABLE proba(\n sym TEXT\n ...


In [6]:
q='''
        SELECT DISTINCT DATE(date)
          FROM prices_d
         WHERE sym='BYND'
           AND DATE(date) >= '{}'
           --AND DATE(date) <= '{}'
         ORDER BY date
'''.format('2021-03-01', '2021-02-06')
df = pd.read_sql(q, db.conn)
df

,DATE(date)
0,2021-03-01


# Manual db view/modify

In [17]:
#execute
assert 0
q = '''
ALTER TABLE trading_days
RENAME COLUMN Date TO date
'''
q = '''
drop table trading_days
'''
q='''
ALTER TABLE prices_interday
  RENAME TO prices_d
'''
q='''
drop index index_prices_interday
'''

q='''
    UPDATE prices_m
       SET is_reg_hours = CASE
                           WHEN time(datetime) < time('09:30:00') THEN 0
                           WHEN time(datetime) > time('15:59:00') THEN 0
                           ELSE 1
                           END
     WHERE DATE(datetime) >= '2020-11-02'
       AND DATE(datetime) <= '2020-11-24'
'''
q='''
    --UPDATE prices_m
       SET datetime = DATETIME(datetime, '-1 hours')
     WHERE DATE(datetime) >= '2020-11-02'
       AND DATE(datetime) <= '2020-11-24'
'''
db.execute(q)
beeps()

AssertionError: 

In [5]:
get_df_prices('BYND', '2020-11-28', '2020-12-03')

,sym,datetime,open,high,low,adj_close,volume,is_reg_hours
133,BYND,2020-11-30 05:21:00,141.900000,141.900000,141.900000,141.900000,0,0
134,BYND,2020-11-30 07:07:00,140.360000,140.360000,140.350000,140.350000,0,0
135,BYND,2020-11-30 08:17:00,140.530000,140.530000,140.530000,140.530000,0,0
136,BYND,2020-11-30 08:18:00,140.900000,140.900000,140.900000,140.900000,0,0
137,BYND,2020-11-30 08:33:00,141.180000,141.250000,141.180000,141.250000,0,0
...,...,...,...,...,...,...,...,...
1066,BYND,2020-12-02 10:50:00,135.070007,135.388702,134.800003,135.070007,11643,1
1067,BYND,2020-12-02 10:51:00,134.860001,135.559998,134.800003,135.490005,14073,1
1068,BYND,2020-12-02 10:52:00,135.489899,135.930206,135.445007,135.930206,10986,1
1069,BYND,2020-12-02 10:53:00,135.759995,136.339996,135.759995,136.294998,14608,1


In [24]:
dt_info = yf.Ticker('GME').info

In [5]:
q = '''
    DELETE
      FROM prices_d 
     WHERE DATE(date) = '{}'
'''.format('2021-03-02')
db.execute(q)

In [25]:
dt_info

{'zip': '76051',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 14000,
 'longBusinessSummary': 'GameStop Corp. operates as a multichannel video game, consumer electronics, and collectibles retailer in the United States, Canada, Australia, and Europe. The company sells new and pre-owned video game platforms; accessories, including controllers, gaming headsets, virtual reality products, and memory cards; new and pre-owned video game software; and in-game digital currency, digital downloadable content, and full-game downloads, as well as network points cards, and prepaid digital and prepaid subscription cards. It also sells collectibles comprising licensed merchandise primarily related to the video game, television, and movie industries, as well as pop culture themes. The company operates its stores and e-commerce sites under the GameStop, EB Games, and Micromania brands; and collectibles stores under the Zing Pop Culture and ThinkGeek brand, as well as offers Game Informer, a prin

In [28]:
def get_df_info(sym):
    '''Returns dataframe containing general info about input symbol
    Args:
        sym (str): e.g.  BYND
    Returns:
        df_info (pandas.DataFrame)
            sym (str)
            long_name (str)
            sec (str)
            ind (str)
            quote_type (str)
            fund_family (str)
            summary (str)
            timestamp (datetime)
    '''
    dt_info = yf.Ticker(sym).info
    dt_info['timestamp'] = datetime.datetime.now()
    dt_info['sector'] = dt_info.get('sector')
    dt_col = {
        'symbol':'sym',
        'longName':'long_name',
        'sector':'sec',
        'industry':'ind',
        'quoteType':'quote_type',
        'fundFamily':'fund_family',
        'longBusinessSummary':'summary',
        'timestamp':'timestamp',
    }
    dt_info = {key:dt_info.get(key) for key in dt_col}
    df_info = pd.DataFrame([dt_info])
    df_info = df_info.rename(columns=dt_col)
    return df_info

test = get_df_info('GME')

In [29]:
test

,sym,long_name,sec,ind,quote_type,fund_family,summary,timestamp
0,GME,GameStop Corp.,Consumer Cyclical,Specialty Retail,EQUITY,None,GameStop Corp. operates as a multichannel vide...,2021-02-14 17:15:06.556169
